In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.argv=['']
del sys
os.chdir("../")

In [2]:
import os
import json
import math
import torch
import numpy
import pandas
import logging
import pandas as pd
import numpy as np
from IPython import embed
from common import data_preprocess 
from common.utils import print_to_json
from common.dataloader import load_SMAP_MSL_dataset, load_CSV_dataset, load_SMD_dataset, load_kddcup_dataset
from collections import defaultdict
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from glob import glob

result_files = glob("exp_results/inter_pca/exp_results_smd_*.csv")
# result_files = glob("exp_results/no-interaction/exp_results_smd_*.csv")
# result_files = glob("exp_results/inter_pca/exp_results_SMAP_*.csv")
# result_files = glob("exp_results/inter_pca/exp_results_MSL_*.csv")
# result_files = glob("exp_results_MSL.csv")
dfs = []
for f in result_files:
    basename = os.path.basename(f).replace(".csv", "").replace("exp_results_", "")
    if "smd" in basename:
        dataset = "_".join(basename.split("_")[0:2])
        feat = "_".join(basename.split("_")[2:])
        dataset_short = dataset.split("-")[0]
    else:
        dataset = basename.split("_")[0]
        feat = "_".join(basename.split("_")[1:])
        dataset_short = ""
    df = pd.read_csv(f)
    df["dataset"] = dataset
    df["dataset_short"] = dataset_short
    df["feat"] = feat
    dfs.append(df)
all_df = pd.concat(dfs).reset_index(drop=True)
all_df

,linkage,inter,norm,F1,Recall,Precision,Anomaly_ratio,dataset,dataset_short,feat
0,complete,True,True,0.000000,0.000000,0.000000,0.001000,smd_3-2,smd_3,inter_norm
1,complete,True,True,0.823328,0.784955,0.865645,0.011184,smd_3-2,smd_3,inter_norm
2,complete,True,True,0.748246,0.658888,0.865645,0.021367,smd_3-2,smd_3,inter_norm
3,complete,True,True,0.685714,0.567711,0.865645,0.031551,smd_3-2,smd_3,inter_norm
4,complete,True,True,0.632411,0.498184,0.865645,0.041735,smd_3-2,smd_3,inter_norm
...,...,...,...,...,...,...,...,...,...,...
16795,average,True,False,0.170515,0.093243,0.995491,0.459265,smd_3-2,smd_3,inter_pca
16796,average,True,False,0.167425,0.091398,0.995491,0.469449,smd_3-2,smd_3,inter_pca
16797,average,True,False,0.164433,0.089618,0.995491,0.479633,smd_3-2,smd_3,inter_pca
16798,average,True,False,0.161593,0.087933,0.995491,0.489816,smd_3-2,smd_3,inter_pca


In [6]:
g_df = all_df.loc[all_df.groupby(['dataset', "feat", "linkage"])['F1'].idxmax().values]
g_df = g_df.groupby(['feat', "linkage"]).mean()
g_df

inter   norm        F1    Recall  Precision  \
feat           linkage                                                 
inter          average    True  False  0.800050  0.828307   0.813279   
               complete   True  False  0.812291  0.843455   0.829622   
               single     True  False  0.766174  0.808004   0.789217   
inter_norm     average    True   True  0.507285  0.500182   0.697531   
               complete   True   True  0.458808  0.462190   0.650727   
               single     True   True  0.523099  0.540221   0.663686   
inter_norm_pca average    True   True  0.437836  0.411842   0.682511   
               complete   True   True  0.443488  0.397528   0.695460   
               single     True   True  0.440512  0.436897   0.660236   
inter_pca      average    True  False  0.474052  0.431967   0.657498   
               complete   True  False  0.526048  0.497150   0.706009   
               single     True  False  0.440641  0.409272   0.670430   

                         Anomaly_ratio  
feat           linkage                  
inter          average        0.013366  
               complete       0.014093  
               single         0.016276  
inter_norm     average        0.055192  
               complete       0.056647  
               single         0.030824  
inter_norm_pca average        0.079923  
               complete       0.096654  
               single         0.074104  
inter_pca      average        0.071922  
               complete       0.061011  
               single         0.091926